patch purpose:

1. drop lts==10
2. for those applicable, fillna with 0
3. replace street type with small population with "other"

In [1]:
from src.constants import fn_features_dc, fn_target_lts_dc, dir_data

In [2]:
import pandas as pd

In [3]:
lts = pd.read_csv(dir_data+fn_target_lts_dc, index_col=0)
lts = lts[lts.LTS != 10].dropna()
lts.to_csv(dir_data+fn_target_lts_dc)

In [4]:
lts.shape

(13293, 1)

In [5]:
seg_attr = pd.read_csv(dir_data+fn_features_dc['seg_attr'],index_col=0)

streettype = seg_attr.STREETTYPE.value_counts()
other = streettype[streettype<=20].index
seg_attr.STREETTYPE = seg_attr.STREETTYPE.apply(lambda x: x if x not in other else 'OTHER')

stype = seg_attr.STREETTYPE.to_frame()
stype.index.name = 'index'
stype.columns = ['name']
stype.to_csv(dir_data + 'seg_street_type.csv')


seg_attr.SEGMENTTYPE = seg_attr.SEGMENTTYPE.astype(str)
seg_attr = pd.get_dummies(seg_attr)
lts.drop('LTS', axis=1).merge(seg_attr, how='left', left_index=True, right_index=True).fillna(0).to_csv(dir_data+fn_features_dc['seg_attr'])

In [7]:
nais0 = ['311','bk_opendc', 'bk_osm', 'poi', 'net_SaE', 'net_SaN', 'v0']

In [8]:
for fn in nais0:
    df = pd.read_csv(dir_data+fn_features_dc[fn], index_col=0)
    df = pd.get_dummies(df)
    if 'index_seg' in df.columns:
        df.set_index('index_seg', inplace=True)
    new_df = lts.drop('LTS', axis=1).merge(
        df, right_index=True, left_index=True, how='left').fillna(0)
    new_df.to_csv(dir_data+fn_features_dc[fn])
#     break

In [9]:
for key, fn in fn_features_dc.items():
    df = pd.read_csv(dir_data+fn, index_col=0)
    if 'index_seg' in df.columns:
        df.set_index('index_seg', inplace=True)
        df.to_csv(dir_data + fn)